In [32]:
#import tensorflow and keras
import tensorflow as tf
from keras import layers, models, datasets, Sequential

#import pandas and matplotlib for accurcy visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [33]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="6Mtasz76njBtqTQgOjjr")
project = rf.workspace("cardetetction").project("car-jgm0v")
version = project.version(2)
dataset = version.download("tensorflow")

loading Roboflow workspace...
loading Roboflow project...


In [71]:
import tensorflow as tf
import pandas as pd
import os

# Load annotations from CSV
def load_annotations(csv_path, images_path):
    annotations = pd.read_csv(csv_path)
    annotations['image_path'] = annotations['filename'].apply(lambda x: os.path.join(images_path, x))
    return annotations

# Parse image and annotations
def parse_image_and_annotations(image_path, xmin, ymin, xmax, ymax, target_size=(224, 224)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_size)

    boxes = tf.convert_to_tensor([[ymin, xmin, ymax, xmax]], dtype=tf.float32)

    return image, boxes

# Load dataset
def load_dataset(images_path, csv_path, target_size=(224, 224)):
    annotations = load_annotations(csv_path, images_path)

    image_paths = annotations['image_path'].values
    xmins = annotations['xmin'].values
    ymins = annotations['ymin'].values
    xmaxs = annotations['xmax'].values
    ymaxs = annotations['ymax'].values

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, xmins, ymins, xmaxs, ymaxs))
    dataset = dataset.map(lambda image_path, xmin, ymin, xmax, ymax:
                          tf.py_function(func=parse_image_and_annotations,
                                         inp=[image_path, xmin, ymin, xmax, ymax, target_size],
                                         Tout=[tf.float32, tf.float32]),
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset

In [72]:

# Load datasets
train_dataset = load_dataset('../content/Car-2/train', '../content/Car-2/train/_annotations.csv')
test_dataset = load_dataset('../content/Car-2/test', '../content/Car-2/test/_annotations.csv')

In [73]:

# Function to batch datasets
def prepare_for_training(ds, batch_size=32, shuffle_buffer_size=1000):
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

batch_size = 32
train_dataset = prepare_for_training(train_dataset, batch_size=batch_size)
test_dataset = prepare_for_training(test_dataset, batch_size=batch_size)

# Example of iterating through the dataset
for images, boxes in train_dataset.take(1):
    print(images.shape)
    print(boxes.shape)

(32, 224, 224, 3)
(32, 1, 4)


In [74]:
model = Sequential()

In [75]:
from keras.layers import Dense
model = models.Sequential([
                          layers.Flatten(input_shape=[224, 224]),
                          layers.Dense(16, activation="relu"),
                          layers.Dense(8, activation="relu"),
                          layers.Dense(10, activation="softmax")
                          ])




In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', run_eagerly=True)

In [76]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dense(128 , activation='relu'),
    tf.keras.layers.Dense(3 , activation='softmax')
])


In [77]:
model.compile(optimizer='adam', loss='categorical_crossentropy', run_eagerly=True)
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 111, 111, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_19 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 54, 54, 64)        0         
 ng2D)                                                           
                                                                 
 dense_33 (Dense)            (None, 54, 54, 128)       8320      
                                                                 
 dense_34 (Dense)            (None, 54, 54, 10)      

In [78]:
model.fit(train_dataset, epochs=100, validation_data=test_dataset)

Epoch 1/100


ValueError: Shapes (32, 1, 4) and (32, 54, 54, 10) are incompatible

In [79]:
for images, labels in train_dataset.take(1):
    print(images.shape, labels.shape)

(32, 224, 224, 3) (32, 1, 4)


In [80]:
import tensorflow as tf
from roboflow import Roboflow

# Initialize Roboflow and download dataset
rf = Roboflow(api_key="6Mtasz76njBtqTQgOjjr")
project = rf.workspace("cardetetction").project("car-jgm0v")
version = project.version(2)
dataset = version.download("tensorflow")

# Load datasets
train_dataset = load_dataset('../content/Car-2/train', '../content/Car-2/train/_annotations.csv')
test_dataset = load_dataset('../content/Car-2/test', '../content/Car-2/test/_annotations.csv')

# Define function to reshape labels
def reshape_labels(labels):
    return tf.squeeze(labels, axis=1)

# Apply reshaping to datasets
train_dataset = train_dataset.map(lambda images, labels: (images, reshape_labels(labels)))
test_dataset = test_dataset.map(lambda images, labels: (images, reshape_labels(labels)))

# Example model definition
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')  # Assuming 10 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Match this with your label format
              metrics=['accuracy'])



loading Roboflow workspace...
loading Roboflow project...


In [83]:
model.fit(train_dataset, epochs=1,batch_size=2, validation_data=test_dataset, verbose=1)

15760/15762 [============================>.] - ETA: 0s

ValueError: Unexpected result of `train_function` (Empty logs). This could be due to issues in input pipeline that resulted in an empty dataset. Otherwise, please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [85]:
def reshape_labels(labels):
    print("Original labels shape:", labels.shape)
    if len(labels.shape) == 3 and labels.shape[1] == 1:
        labels = tf.reshape(labels, [labels.shape[0], labels.shape[2]])
    print("Reshaped labels shape:", labels.shape)
    return labels

# Apply reshaping to datasets
train_dataset = train_dataset.map(lambda images, labels: (images, reshape_labels(labels)))
test_dataset = test_dataset.map(lambda images, labels: (images, reshape_labels(labels)))

# Example model definition with correct input shape
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(640, 640, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # Assuming 10 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' if labels are one-hot encoded
              metrics=['accuracy'])

Original labels shape: <unknown>


ValueError: in user code:

    File "<ipython-input-62-08364d48562a>", line 9, in None  *
        lambda images, labels: (images, reshape_labels(labels))
    File "<ipython-input-84-31f87de67c33>", line 3, in reshape_labels  *
        if len(labels.shape) == 3 and labels.shape[1] == 1:

    ValueError: Cannot take the length of shape with unknown rank.


In [90]:
model.fit(train_dataset, epochs=100, validation_data=test_dataset, verbose=1)

Epoch 1/100
490/493 [============================>.] - ETA: 0s

ValueError: Unexpected result of `train_function` (Empty logs). This could be due to issues in input pipeline that resulted in an empty dataset. Otherwise, please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.